In [35]:
import os # isort:skip # fmt:skip # noqa # nopep8
import sys # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [36]:
from setup_module.imports import *  # isort:skip # fmt:skip # noqa # nopep8


In [37]:
# Transformer variables
method = 'BERT'
results_save_path = f'{models_save_path}{method} Results/'
t = time.time()
n_jobs = -1
n_splits = 10
n_repeats = 3
random_state = 42
refit = True
class_weight = 'balanced'
cv = RepeatedStratifiedKFold(
    n_splits=n_splits, n_repeats=n_repeats, random_state=random_state
)
scoring = 'recall'
scores = [
    'recall', 'accuracy', 'f1', 'roc_auc',
    'explained_variance', 'matthews_corrcoef'
]
scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score),
}
analysis_columns = ['Warmth', 'Competence']
text_col = 'Job Description spacy_sentencized'
metrics_dict = {
    'Mean Cross Validation Train Score': np.nan,
    f'Mean Cross Validation Train - {scoring.title()}': np.nan,
    f'Mean Explained Train Variance - {scoring.title()}': np.nan,
    'Mean Cross Validation Test Score': np.nan,
    f'Mean Cross Validation Test - {scoring.title()}': np.nan,
    f'Mean Explained Test Variance - {scoring.title()}': np.nan,
    'Explained Variance': np.nan,
    'Accuracy': np.nan,
    'Balanced Accuracy': np.nan,
    'Precision': np.nan,
    'Recall': np.nan,
    'F1-score': np.nan,
    'Matthews Correlation Coefficient': np.nan,
    'Fowlkes–Mallows Index': np.nan,
    'ROC': np.nan,
    'AUC': np.nan,
    f'{scoring.title()} Best Threshold': np.nan,
    f'{scoring.title()} Best Score': np.nan,
    'Log Loss/Cross Entropy': np.nan,
    'Cohen’s Kappa': np.nan,
    'Geometric Mean': np.nan,
    'Classification Report': np.nan,
    'Imbalanced Classification Report': np.nan,
    'Confusion Matrix': np.nan,
    'Normalized Confusion Matrix': np.nan
}

# Transformer variables
max_length = 512
returned_tensor = 'pt'
cpu_counts = torch.multiprocessing.cpu_count()
device = torch.device('mps') if torch.has_mps and torch.backends.mps.is_built() and torch.backends.mps.is_available(
) else torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device_name = str(device.type)
print(f'Using {device_name.upper()}')
# Set random seed
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
DetectorFactory.seed = random_state
cores = multiprocessing.cpu_count()
bert_model_name = 'bert-base-uncased'
bert_tokenizer = BertTokenizerFast.from_pretrained(
    bert_model_name, strip_accents=True
)
bert_model = BertForSequenceClassification.from_pretrained(
    bert_model_name
).to(device)
accelerator = Accelerator()
optimizer = AdamW(bert_model.parameters(), lr=3e-5)

# Plotting variables
pp = pprint.PrettyPrinter(indent=4)
tqdm.tqdm.pandas(desc='progress-bar')
tqdm_auto.tqdm.pandas(desc='progress-bar')
tqdm.notebook.tqdm().pandas(desc='progress-bar')
tqdm_auto.notebook_tqdm().pandas(desc='progress-bar')
# pbar = progressbar.ProgressBar(maxval=10)
mpl.use('MacOSX')
mpl.style.use(f'{code_dir}/setup_module/apa.mplstyle-main/apa.mplstyle')
mpl.rcParams['text.usetex'] = True
font = {'family': 'arial', 'weight': 'normal', 'size': 10}
mpl.rc('font', **font)
plt.style.use('tableau-colorblind10')
plt.set_cmap('Blues')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 5000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', '{:.2f}'.format)


Using MPS


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

0it [00:00, ?it/s]

0it [00:00, ?it/s]

# Functions

In [38]:
def show_and_close_plots():
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()


In [39]:
def close_plots():
    plt.clf()
    plt.cla()
    plt.close()


In [40]:
def split_data(df, col, analysis_columns, text_col=text_col):

    train_ratio = 0.75
    test_ratio = 0.10
    validation_ratio = 0.15
    test_split = test_size = 1 - train_ratio
    validation_split = test_ratio / (test_ratio + validation_ratio)

    # Split
    print('='*20)
    print('Splitting data into training, testing, and validation sets:')
    print(f'Ratios: train_size = {train_ratio}, test size = {test_ratio}, validation size = {validation_ratio}')

    df = df.dropna(subset=analysis_columns, how='any')
    df = df.reset_index(drop=True)

    train, test = train_test_split(
        df, train_size = 1-test_split, test_size = test_split, random_state=random_state
    )

    val, test = train_test_split(
        test, test_size=validation_split, random_state=random_state
    )

    X_train = np.array(list(train[text_col].astype('str').values))
    y_train = column_or_1d(train[col].astype('int64').values.tolist(), warn=True)

    X_test = np.array(list(test[text_col].astype('str').values))
    y_test = column_or_1d(test[col].astype('int64').values.tolist(), warn=True)

    X_val = np.array(list(val[text_col].astype('str').values))
    y_val = column_or_1d(val[col].astype('int64').values.tolist(), warn=True)

    class_weights = compute_class_weight(class_weight = 'balanced', classes = [0,1], y = y_train)
    class_weights_ratio = class_weights[0]/class_weights[1]
    class_weights_dict = dict(zip(np.unique(y_train), class_weights))

    print('Done splitting data into training, testing, and validation sets.')

    return (
        train, X_train, y_train,
        test, X_test, y_test,
        val, X_val, y_val,
        class_weights,
        class_weights_ratio,
        class_weights_dict
    )


In [41]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, encoded):
        self.encodings = encodings
        self.encoded = encoded

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx], device=device) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.encoded[idx], device=device)
        return item

    def __len__(self):
        return len(self.encoded)


In [42]:
def encode_data(df, col, analysis_columns, text_col=text_col):

    # Split
    (
        train, X_train, y_train,
        test, X_test, y_test,
        val, X_val, y_val,
        class_weights,
        class_weights_ratio,
        class_weights_dict
    ) = split_data(
        df=df_manual, col=col, analysis_columns=analysis_columns, text_col=text_col
    )

    print('='*20)
    print(f'Encoding training, testing, and validation sets with {bert_tokenizer.__class__.__name__}.from_pretrained using {bert_tokenizer.name_or_path}.')

    bert_label2id = {label: id_ for id_, label in enumerate(set(y_train))}
    bert_id2label = {id_: label for label, id_ in bert_label2id.items()}

    X_train_bert_encodings = bert_tokenizer(
        X_train.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors=returned_tensor
    ).to(device)
    y_train_bert_encoded = [bert_label2id[y] for y in y_train]
    bert_train_dataset = MyDataset(X_train_bert_encodings, y_train_bert_encoded)

    X_test_bert_encodings = bert_tokenizer(
        X_test.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors=returned_tensor
    ).to(device)
    y_test_bert_encoded = [bert_label2id[y] for y in y_test]
    bert_test_dataset = MyDataset(X_test_bert_encodings, y_test_bert_encoded)

    X_val_bert_encodings = bert_tokenizer(
        X_val.tolist(), truncation=True, padding=True, max_length=max_length, return_tensors=returned_tensor
    ).to(device)
    y_val_bert_encoded = [bert_label2id[y] for y in y_val]
    bert_val_dataset = MyDataset(X_val_bert_encodings, y_val_bert_encoded)

    print('Done encoding training, testing, and validation sets.')
    print('='*20)
    print(f'Training set shape: {y_train.shape}')
    print('-'*10)
    print(f'Training set example:\n{X_train[0]}')
    print('-'*10)
    print(f'Training set BERT encodings example:\n{" ".join(bert_train_dataset.encodings[0].tokens[:30])}')
    print('-'*10)
    print(f'Training labels after BERT encoding: {set(y_train_bert_encoded)}')
    print('~'*10)
    print(f'Testing set shape: {y_test.shape}')
    print('-'*10)
    print(f'Testing set example:\n{X_test[0]}')
    print('-'*10)
    print(f'Testing set BERT encodings example:\n{" ".join(bert_test_dataset.encodings[0].tokens[:30])}')
    print('-'*10)
    print(f'Testing labels after BERT encoding: {set(y_test_bert_encoded)}')
    print('~'*10)
    print(f'Validation set shape: {y_val.shape}')
    print('-'*10)
    print(f'Validation set example:\n{X_val[0]}')
    print('-'*10)
    print(f'Validation set BERT encodings example:\n{" ".join(bert_val_dataset.encodings[0].tokens[:30])}')
    print('-'*10)
    print(f'Validation labels after BERT encoding: {set(y_val_bert_encoded)}')
    print('~'*10)
    print(f'Class weights:\nRatio = {class_weights_ratio:.2f} (0 = {class_weights[0]:.2f}, 1 = {class_weights[1]:.2f})')
    print('='*20)

    return (
        train, X_train, X_train_bert_encodings, y_train, y_train_bert_encoded, bert_train_dataset,
        test, X_test, X_test_bert_encodings, y_test, y_test_bert_encoded, bert_test_dataset,
        val, X_val, X_val_bert_encodings, y_val, y_val_bert_encoded, bert_val_dataset,
        bert_label2id, bert_id2label, class_weights, class_weights_ratio, class_weights_dict
    )


In [ ]:
def load_Xy(
    col, results_save_path=results_save_path, method=method, protocol=None,
    data=None,
):
    if protocol is None:
        protocol = pickle.HIGHEST_PROTOCOL
    if data is None:
        data = {}

    print(f'Loading Xy from previous for {col}...')
    # Read all dfs into
    for file_path in glob.glob(f'{results_save_path}*.model'):
        classifier_name = file_path.split(f'{searchcv_save_path}{method} ')[
            1].split(' + ')[1].split(' (Save_protocol=')[0]
        file_name = file_path.split(
            f'{searchcv_save_path}{method} ')[-1].split(path_suffix)[0]
        if 'df_' in file_name and 'cv_results' not in file_name and classifier_name not in ignore_classifiers_list:
            data[file_name] = pd.read_pickle(file_path)

    try:
        # Assign dfs to variables
        df_train_data = data['df_train_data']
        df_test_data = data['df_test_data']
        df_val_data = data['df_val_data']

        # Train data
        X_train = df_train_data['X_train'].values
        y_train = df_train_data['y_train'].values
        # Test data
        X_test = df_test_data['X_test'].values
        y_test = df_test_data['y_test'].values
        # Val data
        X_val = df_val_data['X_val'].values
        y_val = df_val_data['y_val'].values

        print(f'Done loading Xy from previous for {col}!')

        # Get class weights and print info
        (
            train_class_weights, train_class_weights_ratio, train_class_weights_dict,
            test_class_weights_dict, test_class_weights_ratio, test_class_weights_dict
        ) = class_weights_print_Xy(
            X_train, X_test,
            y_train, y_test,
            X_val, y_val
        )

        return (
            X_train, y_train,
            X_test, y_test,
            X_val, y_val,
            train_class_weights, train_class_weights_ratio, train_class_weights_dict,
            test_class_weights_dict, test_class_weights_ratio, test_class_weights_dict
        )
    except Exception:
        print(f'Error loading Xy from previous for {col}!')
        return None


In [43]:
def compute_metrics_with_y_pred(
    y_test, y_test_pred,
    pos_label=None, labels=None, zero_division=None, alpha=None
):

    if pos_label is None:
        pos_label = 1
    if labels is None:
        labels = np.unique(y_test_pred)
    if zero_division is None:
        zero_division = 0
    if alpha is None:
        alpha = 0.1

    # Using y_pred
    explained_variance = metrics.explained_variance_score(y_test, y_test_pred)
    accuracy = metrics.accuracy_score(y_test, y_test_pred)
    balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_test_pred)
    precision = metrics.precision_score(y_test, y_test_pred, pos_label=pos_label, labels=labels, zero_division=zero_division)
    recall = metrics.recall_score(y_test, y_test_pred, pos_label=pos_label, labels=labels, zero_division=zero_division)
    f1 = metrics.f1_score(y_test, y_test_pred, pos_label=pos_label, labels=labels, zero_division=zero_division)
    mcc = metrics.matthews_corrcoef(y_test, y_test_pred)
    fm = metrics.fowlkes_mallows_score(y_test, y_test_pred)
    kappa = metrics.cohen_kappa_score(y_test, y_test_pred, labels=labels)
    gmean_iba = imblearn.metrics.make_index_balanced_accuracy(alpha=alpha, squared=True)(geometric_mean_score)
    gmean = gmean_iba(y_test, y_test_pred)
    report = metrics.classification_report(y_test, y_test_pred, labels=labels, zero_division=zero_division)
    imblearn_report = classification_report_imbalanced(
        y_test, y_test_pred, labels=labels, zero_division=zero_division)
    cm = metrics.confusion_matrix(y_test, y_test_pred, labels=labels)
    cm_normalized = metrics.confusion_matrix(y_test, y_test_pred, normalize='true', labels=labels)

    return (
        explained_variance, accuracy, balanced_accuracy, precision,
        recall, f1, mcc, fm, kappa, gmean, report, cm, cm_normalized
    )


In [44]:
def plot_metrics_with_y_pred(
    y_test, y_test_pred,
    col, vectorizer_name, classifier_name,
    pos_label=None
):
    if pos_label is None:
        pos_label = 1
    # Using y_pred_prob
    # Displays
    close_plots()
    cm_curve = metrics.ConfusionMatrixDisplay.from_predictions(
        y_test, y_test_pred, display_labels=labels
    )
    cm_normalized_curve = metrics.ConfusionMatrixDisplay.from_predictions(
        y_test, y_test_pred, normalize='true', display_labels=labels
    )
    roc_curve = metrics.RocCurveDisplay.from_predictions(
        y_test, y_test_pred, pos_label=pos_label
    )
    pr_curve = metrics.PrecisionRecallDisplay.from_predictions(
        y_test, y_test_pred, pos_label=pos_label
    )
    calibration_curve = CalibrationDisplay.from_predictions(
        y_test, y_test_pred, pos_label=pos_label
    )
    close_plots()

    # Plots
    plots_dict = {
        'Confusion Matrix': cm_curve,
        'Normalized Confusion Matrix': cm_normalized_curve,
        'ROC Curve': roc_curve,
        'Precision-Recall Curve': pr_curve,
        'Calibration Curve': calibration_curve,
    }

    print('=' * 20)
    close_plots()
    print('Plotting metrics with y_pred_prob:')

    for plot_name, plot_ in plots_dict.items():
        print(f'Plotting {plot_name}:')
        fig, ax = plt.subplots()
        ax.set_title(
            f'{str(col)} - {plot_name} {vectorizer_name} + {classifier_name}')
        if plot_name == 'ROC Curve':
            ax.plot([0, 1], [0, 1], 'r--', lw=1)
        plot_.plot(ax=ax)
        show_and_close_plots()
        print('=' * 20)

        # Save Plots
        print(f'Saving {plot_name}...')
        for image_save_format in ['eps', 'png', 'svg']:
            plt.savefig(
                f'{plot_save_path}{method} {str(col)} - {plot_name} {vectorizer_name} + {classifier_name}.{image_save_format}',
                format=image_save_format, dpi=3000, bbox_inches='tight'
            )
        print(f'Saved {plot_name}!')
        print('=' * 20)

    # # Visualisation with plot_metric
    # bc = plot_metric.functions.BinaryClassification(
    #     y_test, y_test_pred, labels=[0, 1])

    # # Figures
    # fig = plt.figure(figsize=(15, 10))
    # plt.subplot2grid((2, 6), (1, 1), colspan=2)
    # bc.plot_confusion_matrix(colorbar=True)
    # plt.subplot2grid((2, 6), (1, 3), colspan=2)
    # bc.plot_confusion_matrix(normalize=True, colorbar=True)
    # plt.subplot2grid(shape=(2, 6), loc=(0, 0), colspan=2)
    # bc.plot_roc_curve()
    # plt.subplot2grid((2, 6), (0, 2), colspan=2)
    # bc.plot_precision_recall_curve()
    # plt.subplot2grid((2, 6), (0, 4), colspan=2)
    # bc.plot_class_distribution()
    # show_and_close_plots()
    # bc.print_report()
    # for image_save_format in ['eps', 'png', 'svg']:
    #     plt.savefig(
    #         f'{plot_save_path}{method} plot_metric Curves {str(col)} - {vectorizer_name} + {classifier_name}.{image_save_format}',
    #         format=image_save_format,
    #         dpi=3000, bbox_inches='tight'
    #     )

    # Heatmap
    print('Plotting Heatmap:')
    close_plots()
    classifications_dict = defaultdict(int)
    for _y_test, _y_test_pred in zip(y_test, y_test_pred):
        if _y_test != _y_test_pred:
            classifications_dict[(_y_test, _y_test_pred)] += 1

    dicts_to_plot = [
        {
            f'True {str(col)} value': _y_test,
            f'Predicted {str(col)} value': _y_test_pred,
            'Number of Classifications': _count,
        }
        for (_y_test, _y_test_pred), _count in classifications_dict.items()
    ]
    df_to_plot = pd.DataFrame(dicts_to_plot)
    df_wide = df_to_plot.pivot_table(
        index=f'True {str(col)} value', 
        columns=f'Predicted {str(col)} value', 
        values='Number of Classifications'
    )

    plt.figure(figsize=(9,7))
    sns.set(style='ticks', font_scale=1.2)
    sns.heatmap(df_wide, linewidths=1, cmap='Purples')    
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    show_and_close_plots()

    return (
        cm_curve, cm_normalized_curve, roc_curve, pr_curve, calibration_curve
    )


In [45]:
def compute_metrics_with_y_pred_prob(
    y_test, y_test_pred_prob,
    pos_label=None
):
    if pos_label is None:
        pos_label = 1

    # Using y_pred_prob
    average_precision = metrics.average_precision_score(y_test, y_test_pred_prob)
    roc_auc = metrics.roc_auc_score(y_test, y_test_pred_prob)
    auc = metrics.auc(fpr, tpr)
    fpr, tpr, threshold = metrics.roc_curve(y_test, y_test_pred_prob, pos_label=1)
    loss = metrics.log_loss(y_test, y_test_pred_prob)
    precision_pr, recall_pr, threshold_pr = metrics.precision_recall_curve(y_test, y_test_pred_prob, pos_label=1)

    return (
        average_precision, roc_auc, auc,
        fpr, tpr, threshold,loss,
        precision_pr, recall_pr, threshold_pr
    )


In [46]:
def compute_metrics(
    predicted_results,
    with_y_pred=None,
    with_y_pred_prob=None
):
    if with_y_pred is None:
        with_y_pred = True
    if with_y_pred_prob is None:
        with_y_pred_prob = False
    
    # Get y_test_pred
    y_test = predicted_results.label_ids
    y_test_pred = [bert_label2id[l] for l in predicted_results.predictions.argmax(-1).flatten().tolist()]

    # # Get y_test_pred_prob
    # try:
    #     y_test_pred_prob = torch.nn.functional.softmax(torch.tensor(predicted_results.predictions, device=device), dim=-1)
    #     print('Using torch.nn.functional.softmax')
    # except Exception:
    #     y_test_pred_prob = scipy.special.softmax(predicted_results.predictions, axis=1)
    #     print('Using scipy.special.softmax')
    # except Exception:
    #     y_test_pred_prob = predicted_results.predictions[:, 1]
    #     print('Using predicted_results.predictions[:, 1]')
    # finally:
    #     y_test_pred_prob = y_test_pred_prob.flatten().tolist()

    # Get metrics
    print('='*20)
    # Using estimator
    if with_estimator:
        print('Computing metrics using estimator.')
        (
            df_cv_score_recall,
            cv_train_scores, cv_test_scores,
            cv_train_recall, cv_test_recall,
            cv_train_explained_variance_recall, cv_test_explained_variance_recall
        ) = compute_metrics_using_estimator(
            estimator, X_test, y_test, col, vectorizer_name, classifier_name,
        )
    # Using y_test_pred
    if with_y_pred:
        print('-'*20)
        print('Computing metrics using y_test_pred.')
        (
            explained_variance, accuracy, balanced_accuracy, precision,
            recall, f1, mcc, fm, kappa, gmean, report, imblearn_report, cm, cm_normalized
        ) = compute_metrics_with_y_pred(
            y_test, y_test_pred, col, vectorizer_name, classifier_name
        )
    # Using y_test_pred_prob
    if with_y_pred_prob:
        print('-'*20)
        print('Computing metrics using y_test_pred_prob.')
        (
            average_precision, roc_auc, auc,
            fpr, tpr, threshold, loss,
            precision_pr, recall_pr, threshold_pr
        ) = compute_metrics_with_y_pred_prob(
            y_test, y_test_pred_prob, col, vectorizer_name, classifier_name,
        )

    # Place metrics into dict
    print('-'*20)
    print('Appending metrics to dict.')
    metrics_dict = {
        # 'Train - Mean Cross Validation Score': float(cv_train_scores),
        # f'Train - Mean Cross Validation - {scoring.title()}': float(cv_train_recall),
        # f'Train - Mean Explained Variance - {scoring.title()}': float(cv_train_explained_variance_recall),
        # 'Test - Mean Cross Validation Score': float(cv_test_scores),
        # f'Test - Mean Cross Validation - {scoring.title()}': float(cv_test_recall),
        # f'Test - Mean Explained Variance - {scoring.title()}': float(cv_test_explained_variance_recall),
        # 'Explained Variance': float(explained_variance),
        'Accuracy': float(accuracy),
        'Balanced Accuracy': float(balanced_accuracy),
        'Precision': float(precision),
        # 'Average Precision': float(average_precision),
        'Recall': float(recall),
        'F1-score': float(f1),
        'Matthews Correlation Coefficient': float(mcc),
        'Fowlkes–Mallows Index': float(fm),
        # 'ROC': float(roc_auc),
        # 'AUC': float(auc),
        # f'{scoring.title()} Best Threshold': threshold,
        # f'{scoring.title()} Best Score': float(best_train_score),
        # 'Log Loss/Cross Entropy': float(loss),
        'Cohen’s Kappa': float(kappa),
        'Geometric Mean': float(gmean),
        'Classification Report': report,
        'Imbalanced Classification Report': np.nan,
        'Confusion Matrix': cm,
        'Normalized Confusion Matrix': cm_normalized
    }

    return metrics_dict

    return metrics_dict


In [47]:
def plot_metrics(
    estimator, X_test, y_test, y_test_pred, y_test_pred_prob,
    col, vectorizer_name, classifier_name, 
    with_y_pred=None, with_y_pred_prob=None
):
    if with_y_pred is None:
        with_y_pred = True
    if with_y_pred_prob is None:
        with_y_pred_prob = False

    # Plotting
    # Using y_test_pred
    if with_y_pred:
        (
            cm_curve, cm_normalized_curve, roc_curve, pr_curve, calibration_curve
        ) = plot_metrics_with_y_pred(
            y_test, y_test_pred,
            col, vectorizer_name, classifier_name, 
        )


In [48]:
def evaluation(
    metrics_dict, df_metrics,
    col, vectorizer_name, classifier_name
):

    # Print metrics
    print('=' * 20)
    print('~' * 20)
    print(' Metrics:')
    print('~' * 20)
    print(f'Classification Report:\n {metrics_dict["Classification Report"]}')
    print('-' * 20)
    for test_metric_name, metric_value in metrics_dict.items():
        if test_metric_name not in ['test_runtime', 'test_samples_per_second', 'test_steps_per_second']:
            metric_name = test_metric_name.split("test_")[1].replace('_', ' ').title()
            if isinstance(metric_name, float):
                print(f'{metric_name}: {round(float(metric_value), 2)}')
            else:
                print(f'{metric_name}: {metric_value}')
            print('-' * 20)

            # Fill Table DF
            if isinstance(metric_value, float):
                df_metrics.loc[
                    (classifier_name), (col, vectorizer_name, metric_name)
                ] = float(metric_value)
            else:
                df_metrics.loc[
                    (classifier_name), (col, vectorizer_name, metric_name)
                ] = str(metric_value)

    print('=' * 20)

    # Plot Metrics
    plot_metrics(
        estimator, X_test, y_test, y_test_pred, y_test_pred_prob,
        col, vectorizer_name, classifier_name, 
    )

    return df_metrics


In [69]:
# Function to place Xy and CV data in df and save
def save_Xy_search_cv_estimator(
    X_train, y_train, y_train_pred,
    X_test, y_test, y_test_pred, y_test_pred_prob,
    X_val, y_val, estimator,
    col, vectorizer_name, classifier_name,
    results_save_path=results_save_path,
    method=method, compression=None, protocol=None,
    path_suffix=None, data=None
):

    if compression is None:
        compression = False
    if protocol is None:
        protocol = pickle.HIGHEST_PROTOCOL
    if path_suffix is None:
        path_suffix = f' - {str(col)} - {vectorizer_name} + {classifier_name}.pkl'
    if data is None:
        data = {}

    # Make df_train_data
    df_train_data = pd.DataFrame(
        {
            'X_train': X_train,
            'y_train': y_train,
            'y_train_pred': y_train_pred,
        },
    )
    # Make df_test_data
    df_test_data = pd.DataFrame(
        {
            'X_test': X_test,
            'y_test': y_test,
            'y_test_pred': y_test_pred,
            'y_test_pred_prob': y_test_pred_prob,
        },
    )
    # Make df_val_data
    df_val_data = pd.DataFrame(
        {
            'X_val': X_val,
            'y_val': y_val,
        },
    )

    # Make data dict
    data['df_train_data'] = df_train_data
    data['df_test_data'] = df_test_data
    data['df_val_data'] = df_val_data
    data['Estimator'] = estimator

    # Save files
    print('='*20)
    print('Saving Xy and estimator...')
    for file_name, file_ in data.items():
        if not isinstance(file_, pd.DataFrame) and file_name == 'Estimator' and 'df_' not in file_name:
            estimator.save_model(f'{results_save_path}{method} {file_name}{path_suffix.split(".pkl")[0]}.model')
            with open(
                f'{results_save_path}{method} {file_name}{path_suffix.split(".pkl")[0]}.model', 'wb'
            ) as f:
                joblib.dump(file_, f, compress=compression, protocol=protocol)
        elif isinstance(file_, pd.DataFrame) and 'df_' in file_name:
            file_.to_pickle(
                f'{results_save_path}{method} {file_name}{path_suffix}', protocol=protocol
            )
    print(f'Done saving Xy and estimator!\n{list(data.keys())}')
    print('='*20)


In [50]:
# Save Model
def save_table(df_metrics, estimator, col, vectorizer_name, classifier_name, table_save_path=table_save_path, save_name=None, save_formats_dict=None):
    if save_name is None:
        save_name = 'Estimators Table'

    # Save metrics df
    print(f'Saving Model and Table for {vectorizer_name} + {classifier_name}.')
    df_metrics.to_csv(f'{table_save_path}{save_name}.csv')
    df_metrics.to_pickle(f'{table_save_path}{save_name}.pkl')
    df_metrics.to_excel(f'{table_save_path}{save_name}.xlsx')
    df_metrics.to_latex(f'{table_save_path}{save_name}.tex')
    df_metrics.to_markdown(f'{table_save_path}{save_name}.md')
    df_metrics.to_html(f'{table_save_path}{save_name}.html')


In [51]:
# Save Model
def get_fitted_estimators(results_save_path=results_save_path, method=method):

    estimators_list = []

    for estimator_path in glob.glob(f'{results_save_path}{method} Estimator - *.pkl'):
        with open(estimator_path, 'rb') as f:
            estimators_list.append(joblib.load(f))

    return estimators_list


In [52]:
def comparison_plots(
    estimators_list, X_test, y_test, col,
    curves_dict=None, cmap=plt.cm.Blues
):

    curves_dict = {
        'ROC Curve': metrics.RocCurveDisplay,
        'Precision Recall Curve': metrics.PrecisionRecallDisplay,
        'Calibration Curve': metrics.CalibrationDisplay,
    }

    assert len(estimators_list) != 0

    for curve_name, curve_package in curves_dict.items():
        print('-' * 20)
        print(f'{str(curve_name)}: {str(col)}')
        fig, ax = plt.subplots()
        ax.set_title(f'{str(curve_name)}: {str(col)}')
        for estimator in estimators_list:
            curve = curve_package.from_estimator(
                estimator, X_test, y_test, pos_label=1, ax=ax,
                name=f'{estimator.steps[0][0]} + {estimator.steps[1][0]} + {estimator.steps[-1][0]}'
            )
        show_and_close_plots()

        # Save Plots
        print('Saving plots.')
        for image_save_format in ['eps', 'png', 'svg']:
            curve.figure_.savefig(
                f'{plot_save_path}{method} {str(col)} - All {str(curve_name)}s.{image_save_format}',
                format=image_save_format,
                dpi=3000, bbox_inches='tight'
            )


# Training

### READ DATA

In [53]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_trainning.pkl').reset_index(drop=True)
# HACK REMOVE THIS!!!!!!
df_manual = df_manual.groupby(analysis_columns).sample(n=200).reset_index(drop = True)


In [54]:
%%time
print('#'*40)
print('Starting!')
print('#'*40)

analysis_columns = ['Warmth', 'Competence']
text_col = 'Job Description spacy_sentencized'

# Load Table DF
# df_metrics = pd.read_pickle(f'{table_save_path}Classifiers Table.pkl')

for col in tqdm.tqdm(analysis_columns):

    assert len(df_manual[df_manual[str(col)].map(df_manual[str(col)].value_counts() > 1)]) != 0, f'Dataframe has no {col} values!'
    print('-'*20)
    print(f'{"="*30} TRAINING DATASET OF LENGTH {len(df_manual)} ON {col.upper()} {"="*30}')
    print('-'*20)

    # Split
    (
        train, X_train, X_train_bert_encodings, y_train, y_train_bert_encoded, bert_train_dataset,
        test, X_test, X_test_bert_encodings, y_test, y_test_bert_encoded, bert_test_dataset,
        val, X_val, X_val_bert_encodings, y_val, y_val_bert_encoded, bert_val_dataset,
        bert_label2id, bert_id2label, class_weights, class_weights_ratio, class_weights_dict
    ) = encode_data(
        df=df_manual, col=col, analysis_columns=analysis_columns, text_col=text_col
    )

    # Load pre-trained BERT model
    bert_model = BertForSequenceClassification.from_pretrained(
        bert_model_name, num_labels=len(bert_id2label)
    ).to(device)

    # Accelerate model
    bert_model, bert_train_dataset, bert_test_dataset, bert_val_dataset = accelerator.prepare(bert_model, bert_train_dataset, bert_test_dataset, bert_val_dataset)
    # bert_model.eval()

    # Initialize BERT Trainer
    print('='*30)
    tokenizer_name = bert_tokenizer.__class__.__name__
    classifier_name = bert_model.__class__.__name__
    print(f'Initializing BERT Trainer using {tokenizer_name} + {classifier_name} for {col}')

    # Set BERT fine-tuning parameters
    bert_training_args = TrainingArguments(
        output_dir=f'{results_save_path}{method} Results',
        logging_dir=f'{results_save_path}{method} Logs',
        seed=random_state,
        torch_compile=True,
        use_mps_device=True,
        optim='adamw_torch',
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=20,
        learning_rate=5e-5,
        warmup_steps=100,
        weight_decay=0.01,
        logging_steps=100,
        evaluation_strategy='steps',
    )

    # Pass data to trainer 
    print('-'*20)
    print('Passing arguments to estimator.')
    with joblib.parallel_backend(backend='loky', n_jobs=n_jobs):
        bert_model, bert_train_dataset, bert_test_dataset, bert_val_dataset = accelerator.prepare(bert_model, bert_train_dataset, bert_test_dataset, bert_val_dataset)
        estimator = Trainer(
            model=bert_model,
            tokenizer=bert_tokenizer,
            args=bert_training_args,
            train_dataset=bert_train_dataset,
            eval_dataset=bert_val_dataset,
            compute_metrics=compute_metrics,
        )
        if estimator.place_model_on_device:
            estimator.model.to(device)

        # Train trainer
        print('-'*20)
        print(f'Starting training for {col}.')
        estimator.train()
        print('Done training!')
        print('-'*20)

        # Save model
        print('-'*20)
        print(f'Saving model for {col}.')
        save_Xy_search_cv_estimator(
            X_train, y_train, y_train_pred,
            X_test, y_test, y_test_pred, y_test_pred_prob,
            X_val, y_val, estimator,
            col, vectorizer_name, classifier_name,
        )
        estimator.save_model(f'{results_save_path}{method} Estimator {str(col)} - {tokenizer_name} + {classifier_name})')
        print('Done training!')
        print('-'*20)

        # Evaluate
        print('-'*20)
        print(f'Evaluating estimator for {col}.')
        estimator.evaluate()
        print('Done evaluating!')

        # Get predictions
        print(f'Getting prediction results for {col}.')
        predicted_results = estimator.predict(bert_test_dataset)
        print(f'Predictions shape for {col}: {predicted_results.predictions.shape}')
        print('-'*20)
    
    # Get y_test_pred
    print('-'*20)
    print(f'Getting y_test_pred from prediction results for {col}')
    y_test = predicted_results.label_ids
    y_test_pred = [bert_label2id[l] for l in predicted_results.predictions.argmax(-1).flatten().tolist()]
    print(f'Length of y_test_pred: {len(y_test_pred)}')
    print('-'*20)

    # Get y_test_pred_prob
    print('-'*20)
    print(f'Getting y_test_pred_prob for {col}')
    try:
        y_test_pred_prob = torch.nn.functional.softmax(torch.tensor(predicted_results.predictions, device=device), dim=-1)
        print('Using torch.nn.functional.softmax')
    except Exception:
        y_test_pred_prob = scipy.special.softmax(predicted_results.predictions, axis=1)
        print('Using scipy.special.softmax')
    except Exception:
        y_test_pred_prob = predicted_results.predictions[:, 1]
        print('Using predicted_results.predictions[:, 1]')
    finally:
        y_test_pred_prob = y_test_pred_prob.flatten().tolist()
    print(f'Length of y_test_pred_prob: {len(y_test_pred_prob)}')
    print('-'*20)
    
    # Examine predictions
    print('-'*20)
    print(f'Examining predictions for {col}')
    print('Incorrectly Classified Reviews:')
    for _y_test, _y_test_pred, _X_test in random.sample(list(zip(y_test, y_test_pred, X_test)), 20):
        if _y_test != _y_test_pred:
            print(f'TRUE LABEL: {_y_test}')
            print(f'PREDICTED LABEL: {_y_test_pred}')
            print(f'REVIEW TEXT: {_X_test[:100]}...')

    # Evluate estimator
    print('-'*20)
    print(f'Probs evaluation and ploting metrics for {col}')
    df_metrics = evaluation(predicted_results, df_metrics, col, tokenizer_name, classifier_name)
    print()

    # Save BERT Model
    print('-'*20)
    print(f'Saving estimator and metrics table for {col}')
    save_table(df_metrics, estimator, col, tokenizer_name, classifier_name)
    print()

    # Compare Estimators
    print('='*20)
    print(f'Comparing estimators for {col}')
    comparison_plots(get_fitted_estimators().append(estimator), X_test, y_test, col)
    print('='*20)
    print()

print('#'*40)
print('DONE!')
print('#'*40)



########################################
Starting!
########################################


  0%|          | 0/2 [00:00<?, ?it/s]

--------------------
============================== TRAINING DATASET OF LENGTH 800 ON WARMTH ==============================
--------------------
Splitting data into training, testing, and validation sets:
Ratios: train_size = 0.75, test size = 0.1, validation size = 0.15
Done splitting data into training, testing, and validation sets.
Encoding training, testing, and validation sets with BertTokenizerFast.from_pretrained using bert-base-uncased.
Done encoding training, testing, and validation sets.
Training set shape: (600,)
----------
Training set example:
Automate) who will be responsible for preparation and execution of improvements working with the RPA and Microsoft Power
----------
Training set BERT encodings example:
[CLS] auto ##mate ) who will be responsible for preparation and execution of improvements working with the r ##pa and microsoft power [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
----------
Training labels after BERT encoding: {0, 1}
~~~~~~~~~~
Testing set shape: (

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Initializing BERT Trainer using BertTokenizerFast + BertForSequenceClassification for Warmth
--------------------
Passing arguments to estimator.
--------------------
Starting training for Warmth.


...

/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/1230619439.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx], device=device) for key, val in self.encodings.items()}
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/1230619439.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx], device=device) for key, val in self.encodings.items()}
/var/folders/46/q15p556n1dd63z6g

{'loss': 0.576, 'learning_rate': 5e-05, 'epoch': 2.63}


  0%|          | 0/6 [00:00<?, ?it/s]

/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/1230619439.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx], device=device) for key, val in self.encodings.items()}
  0%|          | 0/2 [02:44<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/IPython/cor │
│ e/magics/execution.py:1319 in time                                                               │
│                                                                                                  │
│   1316 │   │   else:                                                                             │
│   1317 │   │   │   st = clock2()                                                                 │
│   1318 │   │   │   try:                                                                          │
│ ❱ 1319 │   │   │   │   exec(code, glob, local_ns)                                                │
│   1320 │   │   │   │   out=None                                                                  │
│   1321 │   │   │   │   # multi-line %%time case                                                  │
│   1322 │   │   │   │   if expr_val is not None:                                                  │
│ <timed exec>:80 in <module>                                                                      │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/transformer │
│ s/trainer.py:1633 in train                                                                       │
│                                                                                                  │
│   1630 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1631 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1632 │   │   )                                                                                 │
│ ❱ 1633 │   │   return inner_training_loop(                                                       │
│   1634 │   │   │   args=args,                                                                    │
│   1635 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1636 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/transformer │
│ s/trainer.py:1979 in _inner_training_loop                                                        │
│                                                                                                  │
│   1976 │   │   │   │   │   self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epo  │
│   1977 │   │   │   │   │   self.control = self.callback_handler.on_step_end(args, self.state, s  │
│   1978 │   │   │   │   │                                                                         │
│ ❱ 1979 │   │   │   │   │   self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_k  │
│   1980 │   │   │   │   else:                                                                     │
│   1981 │   │   │   │   │   self.control = self.callback_handler.on_substep_end(args, self.state  │
│   1982                                                                                           │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/transformer │
│ s/trainer.py:2236 in _maybe_log_save_evaluate                                                    │
│                                                                                                  │
│   2233 │   │   │   │   │   │   metric_key_prefix=f"eval_{eval_dataset_name}",                    │
│   2234 │   │   │   │   │   )                               

In [55]:
# Get y_test_pred_prob
print('-'*20)
print(f'Getting y_test_pred_prob for {col}')
try:
    y_test_pred_prob = torch.nn.functional.softmax(torch.tensor(predicted_results.predictions, device=device), dim=-1)
    print('Using torch.nn.functional.softmax')
except Exception:
    y_test_pred_prob = scipy.special.softmax(predicted_results.predictions, axis=1)
    print('Using scipy.special.softmax')
except Exception:
    y_test_pred_prob = predicted_results.predictions[:, 1]
    print('Using predicted_results.predictions[:, 1]')
finally:
    y_test_pred_prob = y_test_pred_prob.flatten().tolist()
print(f'Length of y_test_pred_prob: {len(y_test_pred_prob)}')

--------------------
Getting y_test_pred_prob for Warmth


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3100324511.py:5 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3100324511.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predicted_results' is not defined

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3100324511.py:8 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3100324511.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predicted_results' is not defined

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3100324511.py:14 in <module>    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3100324511.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'y_test_pred_prob' is not defined

In [56]:
type(y_test)


numpy.ndarray

In [57]:
# Examine predictions
print('-'*20)
print(f'Examining predictions for {col}')
print('Correctly Classified Reviews:')
for y_test, _y_test_pred, _X_test in random.sample(list(zip(y_test, y_test_pred, X_test)), 20):
    if y_test == y_test_pred:
        print(f'LABEL: {y_test}')
        print(f'REVIEW TEXT: {_X_test[:100]}...')
        print('-'*20)
        print()

# print('Incorrectly Classified Reviews:')
# for y_test, y_test_pred, _X_test in random.sample(list(zip(y_test, y_test_pred, X_test)), 20):
#     if y_test != y_test_pred:
#         print(f'TRUE LABEL: {y_test}')
#         print(f'PREDICTED LABEL: {y_test_pred}')
#         print(f'REVIEW TEXT: {_X_test[:100]}...')
#         print()

--------------------
Examining predictions for Warmth
Correctly Classified Reviews:


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3618849491.py:5 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3618849491.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'y_test_pred' is not defined

In [58]:
metrics_dict

{'Mean Cross Validation Train Score': nan,
 'Mean Cross Validation Train - Recall': nan,
 'Mean Explained Train Variance - Recall': nan,
 'Mean Cross Validation Test Score': nan,
 'Mean Cross Validation Test - Recall': nan,
 'Mean Explained Test Variance - Recall': nan,
 'Explained Variance': nan,
 'Accuracy': nan,
 'Balanced Accuracy': nan,
 'Precision': nan,
 'Recall': nan,
 'F1-score': nan,
 'Matthews Correlation Coefficient': nan,
 'Fowlkes–Mallows Index': nan,
 'ROC': nan,
 'AUC': nan,
 'Recall Best Threshold': nan,
 'Recall Best Score': nan,
 'Log Loss/Cross Entropy': nan,
 'Cohen’s Kappa': nan,
 'Geometric Mean': nan,
 'Classification Report': nan,
 'Imbalanced Classification Report': nan,
 'Confusion Matrix': nan,
 'Normalized Confusion Matrix': nan}

In [59]:
# Get y_test_pred_proba
print('-'*20)
print(f'Getting y_test_pred_prob for {col}')
try:
    y_test_pred_prob = torch.nn.functional.softmax(torch.tensor(predicted_results.predictions, device=device), dim=-1)
    print('Using torch.nn.functional.softmax')
except Exception:
    y_test_pred_prob = scipy.special.softmax(predicted_results.predictions, axis=1)
    print('Using scipy.special.softmax')
except Exception:
    y_test_pred_prob = predicted_results.predictions[:, 1]
    print('Using predicted_results.predictions[:, 1]')
finally:
    y_test_pred_prob = y_test_pred_prob.flatten().tolist()
print(f'Length of y_test_pred_prob: {len(y_test_pred_prob)}')



--------------------
Getting y_test_pred_prob for Warmth


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/277311591.py:5 in <module>      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/277311591.py'                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predicted_results' is not defined

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/277311591.py:8 in <module>      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/277311591.py'                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predicted_results' is not defined

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/277311591.py:14 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/277311591.py'                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'y_test_pred_prob' is not defined

In [60]:
predicted_results.metrics

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/812427403.py:1 in <module>      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/812427403.py'                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predicted_results' is not defined

In [61]:
# Evluate estimator
print('-'*20)
print(f'Probs evaluation and ploting metrics for {col}')
df_metrics = evaluation(predicted_results, df_metrics, col, classifier_name, tokenizer_name)


--------------------
Probs evaluation and ploting metrics for Warmth


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/1347204323.py:4 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/1347204323.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predicted_results' is not defined

In [62]:
estimator.is_model_parallel


False

In [63]:
estimator.args.n_gpu

1

In [64]:
df_manual['Bert Preditcions'] = df_manual['Job Description spacy_sentencized'].progress_apply(
    lambda sentence: estimator.predict(sent)
    for sent in sentence
    if sent and isinstance(sent, (str, list)) and isinstance(sentence, list)
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/457744598.py:3 in <module>      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/457744598.py'                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'sentence' is not defined

In [65]:
metrics_dict = estimator.evaluate()


/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/1230619439.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx], device=device) for key, val in self.encodings.items()}


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/2849674287.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/2849674287.py'                 │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/transformer │
│ s/trainer.py:2932 in evaluate                                                                    │
│                                                                                                  │
│   2929 │   │   start_time = time.time()                                                          │
│   2930 │   │                                                                                     │
│   2931 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 2932 │   │   output = eval_loop(                                                               │
│   2933 │   │   │   eval_dataloader,                                                              │
│   2934 │   │   │   description="Evaluation",                                                     │
│   2935 │   │   │   # No point gathering the predictions if there are no metrics, otherwise we d  │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/transformer │
│ s/trainer.py:3220 in evaluation_loop                                                             │
│                                                                                                  │
│   3217 │   │   │   │   │   EvalPrediction(predictions=all_preds, label_ids=all_labels, inputs=a  │
│   3218 │   │   │   │   )                                                                         │
│   3219 │   │   │   else:                                                                         │
│ ❱ 3220 │   │   │   │   metrics = self.compute_metrics(EvalPrediction(predictions=all_preds, lab  │
│   3221 │   │   else:                                                                             │
│   3222 │   │   │   metrics = {}                                                                  │
│   3223                                                                                           │
│                                                                                                  │
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3556167990.py:31 in             │
│ compute_metrics                                                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3556167990.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'with_estimator' is not defined

In [66]:
metrics_dict


{'Mean Cross Validation Train Score': nan,
 'Mean Cross Validation Train - Recall': nan,
 'Mean Explained Train Variance - Recall': nan,
 'Mean Cross Validation Test Score': nan,
 'Mean Cross Validation Test - Recall': nan,
 'Mean Explained Test Variance - Recall': nan,
 'Explained Variance': nan,
 'Accuracy': nan,
 'Balanced Accuracy': nan,
 'Precision': nan,
 'Recall': nan,
 'F1-score': nan,
 'Matthews Correlation Coefficient': nan,
 'Fowlkes–Mallows Index': nan,
 'ROC': nan,
 'AUC': nan,
 'Recall Best Threshold': nan,
 'Recall Best Score': nan,
 'Log Loss/Cross Entropy': nan,
 'Cohen’s Kappa': nan,
 'Geometric Mean': nan,
 'Classification Report': nan,
 'Imbalanced Classification Report': nan,
 'Confusion Matrix': nan,
 'Normalized Confusion Matrix': nan}

In [67]:
predicted_results = estimator.predict(bert_val_dataset)

/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/1230619439.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx], device=device) for key, val in self.encodings.items()}


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/238477192.py:1 in <module>      │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/238477192.py'                  │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/transformer │
│ s/trainer.py:3008 in predict                                                                     │
│                                                                                                  │
│   3005 │   │   start_time = time.time()                                                          │
│   3006 │   │                                                                                     │
│   3007 │   │   eval_loop = self.prediction_loop if self.args.use_legacy_prediction_loop else se  │
│ ❱ 3008 │   │   output = eval_loop(                                                               │
│   3009 │   │   │   test_dataloader, description="Prediction", ignore_keys=ignore_keys, metric_k  │
│   3010 │   │   )                                                                                 │
│   3011 │   │   total_batch_size = self.args.eval_batch_size * self.args.world_size               │
│                                                                                                  │
│ /opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/transformer │
│ s/trainer.py:3220 in evaluation_loop                                                             │
│                                                                                                  │
│   3217 │   │   │   │   │   EvalPrediction(predictions=all_preds, label_ids=all_labels, inputs=a  │
│   3218 │   │   │   │   )                                                                         │
│   3219 │   │   │   else:                                                                         │
│ ❱ 3220 │   │   │   │   metrics = self.compute_metrics(EvalPrediction(predictions=all_preds, lab  │
│   3221 │   │   else:                                                                             │
│   3222 │   │   │   metrics = {}                                                                  │
│   3223                                                                                           │
│                                                                                                  │
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3556167990.py:31 in             │
│ compute_metrics                                                                                  │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/3556167990.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'with_estimator' is not defined

In [68]:
predicted_results.metrics


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/2731538659.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/46/q15p556n1dd63z6gkwyh896c0000gn/T/ipykernel_56201/2731538659.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predicted_results' is not defined